In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle

path = '/Users/nemo/hku/mfin7036/nlp-ferrari'

In [4]:
articles = pd.read_csv(path + '/data/locations.csv')
articles = articles[articles['Children'] == False]
articles = articles[articles['Company'] != 'WMT']
articles = articles[articles['Company'] != 'GOOGL']
articles.dropna(inplace=True)

articles.sort_values(by=['Company', 'Date'], inplace=True)
articles.reset_index(drop=True, inplace=True)

In [5]:
articles_arr = pickle.load(open(path + '/data/articles_arr.p', 'rb'))

In [41]:
no_stops = pickle.load(open(path + '/data/no_stops_words.p', 'rb'))
articles_arr = [' '.join(w) for w in no_stops]

In [44]:
len(articles_arr)

2043

In [66]:
v = TfidfVectorizer(max_df=0.9)
tfidf = v.fit_transform(articles_arr)

In [68]:
len(v.get_feature_names_out())
v.get_feature_names_out()

array(['00', '000', '0000', ..., 'åsgard', 'ça', 'élan'], dtype=object)

In [58]:
len(articles[articles['Company'] == 'AAPL'])

45

In [61]:
cosine_similarity(tfidf[0:10], tfidf[75:85])

array([[0.03989611, 0.04674614, 0.04127607, 0.0434393 , 0.04455094,
        0.04903627, 0.06492974, 0.0630256 , 0.06097579, 0.05458639],
       [0.04209665, 0.04668728, 0.04849076, 0.05044233, 0.0462759 ,
        0.05226252, 0.06849314, 0.06411368, 0.06227518, 0.05256367],
       [0.03857398, 0.04195253, 0.04443285, 0.04642041, 0.04506537,
        0.04693962, 0.06914478, 0.06226733, 0.06051319, 0.05611965],
       [0.04093833, 0.04572065, 0.04584904, 0.04964744, 0.04745916,
        0.05162154, 0.07669322, 0.06885272, 0.06459044, 0.06086084],
       [0.03674375, 0.04407226, 0.04317727, 0.04611863, 0.04153473,
        0.04817156, 0.07134834, 0.06480727, 0.05633246, 0.07061053],
       [0.03311029, 0.03876703, 0.03855576, 0.04080894, 0.03736113,
        0.04209359, 0.05625923, 0.06090509, 0.05519702, 0.06136788],
       [0.0411268 , 0.04324169, 0.04382653, 0.0472576 , 0.04548918,
        0.04758585, 0.06333995, 0.06112672, 0.06038548, 0.0511567 ],
       [0.03880718, 0.04232915, 0.0404848

In [67]:
tfidf_df = pd.DataFrame(tfidf.toarray(),columns=v.get_feature_names_out())


In [54]:
tfidf_df.to_csv(path + '/data/tfidf.csv')

In [74]:
price_chg_3m = pd.read_csv(path + '/data/price_chg_3m.csv', index_col=0)

In [76]:
price_chg_3m[price_chg_3m['p_c'] > 0].index

Int64Index([   1,    2,    3,    4,    5,    6,    9,   10,   12,   13,
            ...
            2024, 2025, 2027, 2030, 2032, 2037, 2038, 2039, 2040, 2041],
           dtype='int64', length=1356)

In [77]:
price_chg_3m[price_chg_3m['p_c'] <= 0].index

Int64Index([   0,    7,    8,   11,   17,   18,   19,   21,   27,   31,
            ...
            2022, 2023, 2026, 2028, 2029, 2031, 2033, 2034, 2035, 2036],
           dtype='int64', length=640)

In [111]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] > 0].index]

In [112]:
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= 0].index]

In [113]:
cosine_similarity(up_tfidf, up_tfidf).mean()

0.11301263149107163

In [118]:
t = cosine_similarity(up_tfidf, up_tfidf)

(t.sum() - len(t)) / len(t) / (len(t) - 1)

0.11235802826707979

In [95]:
cud = cosine_similarity(up_tfidf, down_tfidf)

In [109]:
cud.mean()

0.11282562386262464

In [119]:
t = cosine_similarity(down_tfidf, down_tfidf)

(t.sum() - len(t)) / len(t) / (len(t) - 1)

0.11346503567339367

In [91]:
t

array([[1.        , 0.73889245, 0.7378461 , ..., 0.07788288, 0.08432978,
        0.0762031 ],
       [0.73889245, 1.        , 0.79971079, ..., 0.06615337, 0.07384252,
        0.07462311],
       [0.7378461 , 0.79971079, 1.        , ..., 0.06435588, 0.0788586 ,
        0.07449311],
       ...,
       [0.07788288, 0.06615337, 0.06435588, ..., 1.        , 0.59688709,
        0.57117413],
       [0.08432978, 0.07384252, 0.0788586 , ..., 0.59688709, 1.        ,
        0.66267386],
       [0.0762031 , 0.07462311, 0.07449311, ..., 0.57117413, 0.66267386,
        1.        ]])

In [122]:
price_chg_3m['p_c'].quantile([0.1, 0.25, 0.3333, 0.5, 0.6666, 0.75, 0.9])

0.1000   -0.090225
0.2500   -0.021786
0.3333    0.003267
0.5000    0.046141
0.6666    0.091554
0.7500    0.117090
0.9000    0.193576
Name: p_c, dtype: float64

In [123]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.193576].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= -0.090225].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11075460358153569
dd 0.10697891406828433
ud 0.10864772492695547


In [108]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.117090].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= -0.021786].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.10802484943470686
dd 0.11128609168985125
ud 0.10886081510936475


In [120]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.091554].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= 0.003267].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11019258740489096
dd 0.11370992643594001
ud 0.11124588101567941


In [124]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.046141].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] < 0.046141].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.1116162053577416
dd 0.11423791783669547
ud 0.11242426038903179


In [121]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] < 0].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11235802826707979
dd 0.11346503567339367
ud 0.11282562386262464
